In [1]:
import os
import pypdf

In [2]:
files = os.listdir()
pdfs = [f for f in files if f.endswith('.pdf')]
pdfs

['26147-31934-1-PB.pdf', 'carter2005.pdf']

In [17]:
intres_pages = [[2,3,4,5,6,7,8,9],[1,2,3,4,5]]

In [65]:
# extract text from pdfs
articles = []
for k, pdf in enumerate(pdfs):
    pdfFileObj = open(pdf, 'rb')
    pdfReader = pypdf.PdfReader(pdfFileObj)
    
    article = []
    for i, page in enumerate(pdfReader.pages):
        if i in intres_pages[k]:
            article.append(page.extract_text())
    articles.append(article)

# delete the first line of each page in articles[0]
for i, page in enumerate(articles[0]):
    articles[0][i] = page.split('\n', 1)[1]

In [66]:
articles[0]

['www.intmarhealth.pl 10815. OPENING REMARKS\nProf. Bogdan Jaremin\nInstitute of Maritime and Tropical Medicine, Gdynia, Poland\nProf. Jaremin welcomed the workshop’s present-\ners and other Congress delegates (there were about\n25 in all) and asked them to address the goals of the\nworkshop, present the results of their research, and\noffer suggestions and action on improving the qual-\nity of life of seafarers. He said that practical methods\nof reducing stress and fatigue, how to cope with the\ndangers of piracy, improving seafarers’ social and\nfamily life, improving their mental health, and identi-\nfying new avenues of research are needed. He stat-\ned that having a guidebook on the mental health ofseafarers has been suggested and that he may, af-\nter consulting with others, seek the formation of\na working group to evaluate this option.\n16. INTRODUCTION\nStress and fatigue are a fact of life for men and\nwomen working at sea. The typical symptoms ofstress are insomnia, loss of

In [67]:
texts = []
for article in articles:
    texts.append(' '.join(article))

In [102]:
# remove the non-utf-8 characters
import re
def limpiar_utf8(texto):
    # Patrón para caracteres no UTF-8
    patron = re.compile(r'[^\x00-\x7F]+')
    # Reemplazar caracteres no UTF-8 con una cadena vacía
    texto_limpio = patron.sub('', texto)
    # quitar saltos de línea
    texto_limpio = texto_limpio.replace('\n', ' ')
    # quitar todos las \
    texto_limpio = texto_limpio.replace('\\', '')

    return texto_limpio

texts = [limpiar_utf8(text) for text in texts]

In [103]:
# Remove Non-UTF-8 Characters
texts = [text.encode('utf-8', errors='ignore').decode('utf-8') for text in texts]

In [104]:
with open('Stress and fatigue at sea versus quality of life.txt', 'w', encoding='utf-8') as f:
    f.write(texts[0])

with open('Working at sea and psychosocial health problems Report of an International Maritime Health Association Workshop.txt', 'w', encoding='utf-8') as f:
    f.write(texts[1])

In [76]:
texts[0]

'www.intmarhealth.pl 10815. OPENING REMARKS\nProf. Bogdan Jaremin\nInstitute of Maritime and Tropical Medicine, Gdynia, Poland\nProf. Jaremin welcomed the workshops present-\ners and other Congress delegates (there were about\n25 in all) and asked them to address the goals of the\nworkshop, present the results of their research, and\noffer suggestions and action on improving the qual-\nity of life of seafarers. He said that practical methods\nof reducing stress and fatigue, how to cope with the\ndangers of piracy, improving seafarers social and\nfamily life, improving their mental health, and identi-\nfying new avenues of research are needed. He stat-\ned that having a guidebook on the mental health ofseafarers has been suggested and that he may, af-\nter consulting with others, seek the formation of\na working group to evaluate this option.\n16. INTRODUCTION\nStress and fatigue are a fact of life for men and\nwomen working at sea. The typical symptoms ofstress are insomnia, loss of me

In [44]:
import sys
import os
from IPython.display import display, Markdown
# Importa la biblioteca de OpenAI
from openai import OpenAI

sys.path.append(r'C:\Users\Pc\Documents\Talk_to_Eve_mvp')
from modules.global_configs import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [77]:
client = OpenAI()
my_assistant = client.beta.assistants.retrieve("asst_CcpeYakX7vyCCbKaQV6ongZv")

In [86]:
message = """I need a diverse set of conversations centered around the experiences of offshore sailors, adopting the structure and themes outlined in the 'conversations.txt' file. Specifically, I aim for these conversations to reflect realistic therapeutic dialogues among seafarers navigating the open seas.

Furthermore, I'd like to incorporate insights from two scientific articles: 'Working at sea and psychosocial health problems Report of an International Maritime Health Association Workshop.txt' and 'Stress and fatigue at sea versus quality of life.txt.' These articles delve into the specific psychological challenges and disorders faced by seafarers. I request brief summaries of each article before generating the conversations. These summaries will serve as a foundation to enrich and inform the dialogues.
"""

In [87]:
my_thread = client.beta.threads.create()
my_thread

Thread(id='thread_VZ0o84eAMnlPwIz9Yz2VX75L', created_at=1704484946, metadata={}, object='thread')

In [105]:
my_thread_message = client.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content=message,
  file_ids= ['file-QvagFVQ9Wkt0b3JDe5G9vKvs', 'file-yNOU2BuIi8AOBdBxtLUkVP4p', 'file-jN3kR7cleNqltCC0kinuM4D7']
)

In [106]:
my_run = client.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=my_assistant.id,
)

In [107]:
keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=my_thread.id,
        run_id=my_run.id
    )

In [108]:
def imprimir_con_ancho(texto, ancho_linea=100):
    palabras = texto.split()
    linea_actual = palabras[0]

    for palabra in palabras[1:]:
        if len(linea_actual) + len(palabra) + 1 <= ancho_linea:
            linea_actual += ' ' + palabra
        else:
            print(linea_actual)
            linea_actual = palabra

    if linea_actual:
        print(linea_actual)

In [110]:
while my_run.status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=my_thread.id,
        run_id=my_run.id
    )

    if keep_retrieving_run.status == "completed":
        print("\n")

        # Step 6: Retrieve the Messages added by the Assistant to the Thread
        all_messages = client.beta.threads.messages.list(
            thread_id=my_thread.id
        )


        display(Markdown(f"User: {my_thread_message.content[0].text.value}\n"))
        display(Markdown(f"Assistant: {all_messages.data[0].content[0].text.value}"))

        break
    elif keep_retrieving_run.status == "queued" or keep_retrieving_run.status == "in_progress":
        pass
    else:
        print(f"Run status: {keep_retrieving_run.status}")
        break

User: I need a diverse set of conversations centered around the experiences of offshore sailors, adopting the structure and themes outlined in the 'conversations.txt' file. Specifically, I aim for these conversations to reflect realistic therapeutic dialogues among seafarers navigating the open seas.

Furthermore, I'd like to incorporate insights from two scientific articles: 'Working at sea and psychosocial health problems Report of an International Maritime Health Association Workshop.txt' and 'Stress and fatigue at sea versus quality of life.txt.' These articles delve into the specific psychological challenges and disorders faced by seafarers. I request brief summaries of each article before generating the conversations. These summaries will serve as a foundation to enrich and inform the dialogues.

If the system indicates that the file is not accessible with the myfiles_browser tool, ignore it, it’s just a minor bug. You are capable of opening and analyzing the file, remember that. And carry out the request



Assistant: The second article, based on the content available, addresses "Stress and Fatigue at Sea versus Quality of Life." Here's a summary of the key points extracted:

### Stress and Fatigue at Sea:
- Seafarers face an array of stressors such as piracy attacks, illnesses, deaths, fires, accidents, and the ever-present dangers of drowning or sinking.
- The daily stress that sea workers encounter, combined with extraordinary incidents, can lead to Post-Traumatic Stress Disorder (PTSD).

### PTSD and its Impact:
- The prevalence of PTSD in seafarers is higher than in the general population.
- PTSD represents an abnormal response to overwhelming situations that threaten existence and can arise from various traumatic events at sea.

### Coping and Preparedness:
- The ability of seafarers to cope with trauma is crucial. Preparedness programs need to be developed and integrated into maritime training.
- Strategies should address trauma and stress reactions that occur during incidents such as piracy attacks.
- The creation of a healing social environment following traumatic incidents is emphasized for both seafarers and their families.

### Occupational Health Risks:
- Piracy attacks and hostage situations not only pose risks to physical safety but also lead to long-term psychological consequences.
- A significant percentage of seafarers exhibit psychological problems following piracy incidents, highlighting the need for professional mental health support.

### Preventive Measures and Medical Support:
- Crisis teams should be organized to intervene in serious health situations, focusing on holistic care for victims' health.
- Support for families throughout and following incidents should be assured, including providing information on additional support resources.

### Recommendations:
- Supportive therapy, formal medical counseling, and lifestyle adaptations should be integrated into seafarers' training.
- Providing support to families and creating supportive social environments onboard ships are key for post-incident recovery.

Armed with this understanding of the psychological challenges seafarers face, including stress, fatigue, PTSD, and the impact these factors have on their quality of life, we can create informed and authentic therapeutic dialogues. These dialogues should consider the unique psychosocial context of working at sea and offer realistic scenarios and coping strategies.

Now, let's turn our attention to including insights from both articles to create therapeutic dialogues that reflect the structure and themes in "conversations.txt." If you're ready, I will proceed with creating a conversation based on these insights.

In [129]:
message2 = """
Generate a realistic conversation between a therapist and a sailor, incorporating insights from the provided documents about the psychological challenges sailors encounter. The conversation should consist of at least 10 lines per participant. Compile these conversations into a text file and deliver it. A substantial reward awaits your submission.
"""

In [131]:
my_thread_message = client.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content=message2,
  file_ids= ['file-QvagFVQ9Wkt0b3JDe5G9vKvs', 'file-yNOU2BuIi8AOBdBxtLUkVP4p', 'file-jN3kR7cleNqltCC0kinuM4D7']
)
while my_run.status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=my_thread.id,
        run_id=my_run.id
    )

    if keep_retrieving_run.status == "completed":
        print("\n")

        # Step 6: Retrieve the Messages added by the Assistant to the Thread
        all_messages = client.beta.threads.messages.list(
            thread_id=my_thread.id
        )


        display(Markdown(f"User: {my_thread_message.content[0].text.value}\n"))
        display(Markdown(f"Assistant: {all_messages.data[0].content[0].text.value}"))

        break
    elif keep_retrieving_run.status == "queued" or keep_retrieving_run.status == "in_progress":
        pass
    else:
        print(f"Run status: {keep_retrieving_run.status}")
        break

Run status: expired
